# **Mount**

Mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install import-ipynb
%cd "/content/gdrive/MyDrive/progetto/code/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/progetto/code


# **Import**

Import the LSH_Index file,necessary to use the index

In [ ]:
import import_ipynb
import pickle
import numpy as np
from LSH_Index import *

importing Jupyter notebook from LSH_Index.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Pickles**

Import the pickles,previously saved,belonging to the finetuned index and the testing dataset

In [ ]:

pick_path = '/content/gdrive/MyDrive/progetto/pickles/lsh_models'
test_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning.npy"
label_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning.npy"

directory = "/definitive/"

pickle_name = "finetuned(5,5).pickle"

print("Reading pickle..." )
LSH = pickle.load( open( pick_path + directory + pickle_name, "rb" ) )

print("pickle loaded.")

print("Loading features and labels...")
test_features = np.load(  test_path  )
test_labels = np.load( label_path )
print("features loaded.")

Reading pickle...
pickle loaded.
Loading features and labels...
features loaded.


# **Compute AP at K**

## Average precision at K 

$AP@k= \frac{1}{N(k)} \sum_{i=1}^{k} \frac{TPseen(i)}{i}$


$TP = true positives$

$N(k) = min(k,TPtotal)$

*TPseen(i) = {0 if i°=FALSE; 1 if i°=TRUE}*

## Mean Average Precision at K

The mAP@k is the mean of AP@K obtained from different queries

$mAP@k= \frac{1}{N} \sum_{i=1}^{N} AP@k_i$

In [ ]:
#compute AP@k
def compute_ap_at_k(ranked_relevance, k):
  #ranked_relevance is a boolean array of dim k. true if retrieved element is relevant for the query, false if not relevant 
  total_relevant_items = ranked_relevance.sum()

  # Precision@i for each query, i.e. the number of relevant items @i divided by i
  #cumsum [1,3,2]=>[1,4,6]
  prec_at_i = ranked_relevance.cumsum() / np.arange(1, k + 1) 

  # sum of prec_at_i only for relevant, divide for total number of relevant items 
  aps =  (ranked_relevance * prec_at_i).sum() / ( total_relevant_items + np.finfo(float).eps ) 
  #print(aps)

  return aps

def retrieve_query_label(results):
  label_results = []
  for result in results:
    label_results.append(result[1])
  return label_results

# **Compute APs**

For all the features belonging to the testing dataset, we retrieve using the index the 10 most similar images. Of this 10 images we retrieve their labels, and we use them in order to compute the ap; the ap is then appended in an array in the position corresponding to the label of the searched feature

In [ ]:
import time

aps = [[] for _ in range(7)]

print("Start testing...")
index = 0

for i in range(len(test_features)):
  index = 0
  #print(i)
  for g in [5]:
    for h in [5]:
      results = LSH.search( test_features[i], g, h,10)
      labels = retrieve_query_label( results )

      ranked_relevance= []
      for label in labels:
        if test_labels[i] == label:
          res = 1
        else:
          res = 0

        ranked_relevance.append(res)

      ranked_relevance = np.array(ranked_relevance)

      ap_k = compute_ap_at_k(ranked_relevance, len(ranked_relevance) ) 
      aps[int(test_labels[i])].append(ap_k)  

      index += 1 
 
print("Testing finished.")


Start testing...
Testing finished.


# **From Label To Folder**

For readibility, we translate the label to the relative folder name

In [ ]:
def from_label_to_folder(label):
    folder = ''

    if label == 0:
      folder = 'Angry'
    elif label == 1:
      folder = 'Disgust'
    elif label == 2:
      folder = 'Fear'
    elif label == 3:
      folder = 'Happy'
    elif label == 4:
      folder = 'Neutral'
    elif label == 5:
      folder = 'Sad'
    elif label == 6:
      folder = 'Surprise'
    elif label == 7:
      folder = 'Z_mirflickr'
    
    return folder
    

# **mAP By Class**

The array will contain, in each position, all the aps belonging to the same class. We compute the mAP and print it,together with additional informations

In [ ]:
map = []

print("Evaluating map class by class...")
for index in range(7):  
  ap = aps[index]
  map.append( np.mean( ap ) )
  
print("map evaluated")

for index in range(7):
  print("Class: "+from_label_to_folder(index)+" , Label: "+str(index)+" , count: " + str(len(aps[index])) +", mAP = "+str(map[index]))


Evaluating map class by class...
map evaluated
Class: Angry , Label: 0 , count: 467, mAP = 0.4898842204817528
Class: Disgust , Label: 1 , count: 56, mAP = 0.3542942176870748
Class: Fear , Label: 2 , count: 496, mAP = 0.38840273269552733
Class: Happy , Label: 3 , count: 895, mAP = 0.7766357018288382
Class: Neutral , Label: 4 , count: 607, mAP = 0.5269859748616025
Class: Sad , Label: 5 , count: 653, mAP = 0.47217706993749803
Class: Surprise , Label: 6 , count: 415, mAP = 0.7053546191568997
